This jupyter notebook shows the creation of a proof of principle Neural Network which trains on the past 5 moves and tries to predict what the next move will be in the rock paper scissors game. Future plans are to create a LSTM network due to their proficiency at working with sequential data - like rock paper scissor rounds. This model has attained a training accuracy of ~50%, but validation around 38%, indicating that the model is overfitting, but still slightly better than simply random guessing. But further exploration into this model is needed as the nature of the data is in fact very *complex*.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/RPS_data_processed.csv')
df = df.rename(columns={'move': 'winning_move'})

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_player2_move(row):
    if row['winner'] == 1:  # Player 1 wins
        if row['move'] == 1:
            return -1
        elif row['move'] == 0:
            return 1
        else:  # scissors
            return 0
    elif row['winner'] == -1:  # Player 2 wins
        return row['move']
    else:  # Draw
        return row['move']

def prepare_sequences(df, sequence_length=5):
    # Add player 2's move to the dataframe
    df['player2_move'] = df.apply(get_player2_move, axis=1)

    X = []
    y = []

    for i in range(len(df) - sequence_length):
        sequence = df.iloc[i:i+sequence_length]
        next_move = df.iloc[i+sequence_length]['player2_move']

        X.append(sequence[['winner', 'move']].values.flatten())
        y.append(next_move)

    return np.array(X), np.array(y)

df = pd.read_csv('/content/RPS_data_processed.csv')
X, y = prepare_sequences(df)

# Encode the moves for y
move_encoder = LabelEncoder()
y_encoded = move_encoder.fit_transform(y)

# Encode the input sequences
input_encoder = LabelEncoder()
X_encoded = input_encoder.fit_transform(X.ravel()).reshape(X.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Unique moves: {move_encoder.classes_}")
print(f"Number of classes: {len(move_encoder.classes_)}")

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# Convert to categorical
num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_cat, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

nn_predictions = model.predict(X_test)
nn_accuracy = np.mean(np.argmax(nn_predictions, axis=1) == y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

new_model = tf.keras.models.load_model('simpleNN.h5')

# Get predictions
y_pred = np.argmax(model.predict(X_test), axis=1)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=move_encoder.classes_,
            yticklabels=move_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()